## Tool-Calling LLM

แนวคิด: ให้โมเดล “รู้จัก” เครื่องมือ (tools) หลายตัว แล้ว โมเดลตัดสินใจเอง ว่าจะเรียกตัวไหน/ไม่เรียกเลย ผ่านการผูกสคีมาด้วย .bind_tools([...])

- Tool binding: model.bind_tools(tools_list) ส่งสคีมาของ tools ให้โมเดลทราบ เพื่อที่เวลาเรียกใช้ โมเดลจะคืนชื่อเครื่องมือ + arguments (tool call) มาให้เรา execute จริง ๆ แล้วเราค่อยส่งผล (observation) กลับเข้าโมเดลรอบถัดไปหากต้องการ 
LangChain
+1

- ยืดหยุ่น: โมเดลอาจเลือก “ไม่เรียก tool”, “เรียก 1 ตัว”, หรือ “หลายตัว” ขึ้นกับโจทย์/พรอมป์ต
- เหมาะกับงาน “ครั้งเดียวจบ” (one-shot)

## Create Tools function

In [ ]:
from langchain.agents import Tool

def calculate_math_tool(expression: str):
    """Calculate mathematical expressions safely"""
    try:
        allowed_chars = set('0123456789+-*/(). ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"result: {expression} = {result}"
        else:
            return "Error: Invalid expression. Only numbers and the symbols +, -, *, /, (, ) are allowed."
    except Exception as e:
        return f"Calculation error : {str(e)}"

calculate_tool = Tool(
    name="calculator",
    func=calculate_math_tool,
    description="Use this to calculate mathematical expressions. Input should be a valid math expression like '2 + 3 * 4' or '(10 + 5) / 3'"
)

calculate_tool.invoke("1+2")

'ผลลัพธ์: 1+2 = 3'

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run
)

## Build Tools

In [ ]:
tools = [calculate_tool, search_tool]

## Bind tools to LLMs

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gpt-oss")

agent = llm.bind_tools(tools)

## Test

In [ ]:
msg1 = agent.invoke("บวก 12 กับ 7 แล้วบอกคำตอบ")
print(msg1)
print(msg1.content)

content='12\u202f+\u202f7\u202f=\u202f19' additional_kwargs={} response_metadata={'model': 'gpt-oss', 'created_at': '2025-08-29T06:31:07.567759Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5624104375, 'load_duration': 182553542, 'prompt_eval_count': 191, 'prompt_eval_duration': 3778954708, 'eval_count': 75, 'eval_duration': 1661844959, 'model_name': 'gpt-oss'} id='run--bcb96e32-b2a4-4518-be78-54aced76caed-0' usage_metadata={'input_tokens': 191, 'output_tokens': 75, 'total_tokens': 266}
12 + 7 = 19


In [ ]:
msg2 = agent.invoke("เมืองหลวงของประเทศไทยคือ?")
print(msg2)
print(msg2.content)

content='เมืองหลวงของประเทศไทยคือ **กรุงเทพมหานคร** (Bangkok)' additional_kwargs={} response_metadata={'model': 'gpt-oss', 'created_at': '2025-08-29T06:31:09.208881Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1626541958, 'load_duration': 77556292, 'prompt_eval_count': 186, 'prompt_eval_duration': 99178584, 'eval_count': 71, 'eval_duration': 1449357875, 'model_name': 'gpt-oss'} id='run--f8657441-ef09-4b9d-92cb-00434ae2592a-0' usage_metadata={'input_tokens': 186, 'output_tokens': 71, 'total_tokens': 257}
เมืองหลวงของประเทศไทยคือ **กรุงเทพมหานคร** (Bangkok)
